# Load packages

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

from os import path
from tensorflow.keras.utils import image_dataset_from_directory
from matplotlib import pyplot as plt
from tensorflow.keras import layers, losses, optimizers, metrics, callbacks, Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
tf.__version__

'2.10.1'

In [4]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 224

# Load data

In [5]:
dir_path = path.join(os.getcwd(), '..', 'cifar-10', 'train')
dataset = image_dataset_from_directory(
    directory=dir_path,
    label_mode=None,
    shuffle=False,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512
)
dataset = dataset.map(lambda x: preprocess_input(x))

Found 50000 files belonging to 1 classes.


# ResNet - initialization

In [6]:
os.makedirs("weights", exist_ok=True)

In [7]:
resnet = ResNet50(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
x = layers.GlobalAveragePooling2D()(resnet.output)
model = Model(resnet.input, x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

# ResNet - output

In [9]:
outputs = [model.predict_on_batch(batch) for batch in dataset]
len(outputs)

98

In [10]:
outputs = np.concatenate(outputs)
outputs.shape

(50000, 2048)

In [11]:
np.save("resnet_output", outputs)